In [1]:
import pandas as pd

In [2]:
hprc_biosample = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/data/production/hprc-production-biosample-table.tsv',sep='\t')
hprc_biosample.rename(columns={'Sample':'sample_id'},inplace=True)

In [42]:
hic_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/submissions/HIC_Y3_fillin_and_Y4_part1-redo/HIC_Y3_fillin_and_Y4_part1-redo-sample_file_hifi_qc_aggregate.csv')

hic_df['sample_id'] = [sample_id.split('-')[0] for sample_id in hic_df['file_name'].tolist()]

hic_sra_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/submissions/HIC_Y3_fillin_and_Y4_part1-redo/sra_biosample_HIC_Y3_fillin_and_Y4_part1.csv')

release = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/submissions/HIC_Y3_fillin_and_Y4_part1-redo/HIC_Y3_fillin_and_Y4_part1-redo.transfer.csv',header=None)

release['file_name'] = [filename.split('/')[-1] for filename in release[1].tolist()]
release.rename(columns={1:'path'},inplace=True)
release.drop(columns=[0],inplace=True)

In [43]:
hic_df = pd.merge(hic_df, release, on='file_name', how='inner')

In [44]:
hic_df.head()

,file_name,total_bp,coverage,ntsm_score,ntsm_result,sample_id,path
0,HG01975-2_S4_L002_R2_001.fastq.gz,45449986488,14.66,0.304342,Similar,HG01975,s3://human-pangenomics/working/HPRC/HG01975/ra...
1,HG03834-2_S8_L003_R2_001.fastq.gz,69883486748,22.54,0.309635,Similar,HG03834,s3://human-pangenomics/working/HPRC/HG03834/ra...
2,HG02129-1_S2_L003_R2_001.fastq.gz,124595417314,40.19,0.309858,Similar,HG02129,s3://human-pangenomics/working/HPRC/HG02129/ra...
3,HG00408-1_S4_L002_R2_001.fastq.gz,75586775321,24.38,0.311182,Similar,HG00408,s3://human-pangenomics/working/HPRC/HG00408/ra...
4,HG02514-1_S5_L004_R2_001.fastq.gz,103145633415,33.27,0.320622,Similar,HG02514,s3://human-pangenomics/working/HPRC/HG02514/ra...


In [45]:
# number of qc samples
hic_df.shape

(344, 7)

In [46]:
hic_df['sample_id'].nunique()

58

In [47]:
# inner merge on hprc biosamples
hic_df = pd.merge(hic_df, hprc_biosample[hprc_biosample['sample_id'].isin(hic_df['sample_id'])], on='sample_id', how='inner')

In [52]:
# poor qc samples
# hic_df[~hic_df['file_name'].isin(hic_sra_df['filename'].tolist())]

In [53]:
hic_sra_df = hic_sra_df[hic_sra_df['filename'].isin(hic_df['file_name'])]
hic_sra_df['common_id'] = hic_sra_df['filename'].str.extract(r'(^.+_L\d{3})')

/data/tmp/ipykernel_1511726/651745410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hic_sra_df['common_id'] = hic_sra_df['filename'].str.extract(r'(^.+_L\d{3})')


In [55]:
df_R1 = hic_sra_df[hic_sra_df['filename'].str.contains('_R1_')].copy()
df_R2 = hic_sra_df[hic_sra_df['filename'].str.contains('_R2_')].copy()

In [56]:
hic_sra_merge_df = pd.merge(df_R1, df_R2, on='common_id', how='inner')

In [57]:
hic_sra_merge_df.shape

(172, 35)

In [58]:
hic_sra_merge_df = hic_sra_merge_df.loc[:, ~hic_sra_merge_df.columns.str.endswith('_y')]
hic_sra_merge_df.columns = hic_sra_merge_df.columns.str.replace('_x$', '', regex=True)

In [59]:
hic_sra_merge_df['library_ID'] = hic_sra_merge_df['library_ID'].str.extract(r'(.+_L\d{3})')

In [60]:
hic_sra_merge_df.drop(columns=['common_id'],inplace=True)

In [61]:
hic_sra_merge_df

,filename,filetype,library_ID,library_strategy,library_source,platform,instrument_model,design_description,generator_facility,generator_contact,assembly,biosample_accession,title,data_type,library_layout,library_selection,shear_method
0,HG00126-1_FC1_S3_L004_R1_001.fastq.gz,fastq,HG00126-1_FC1_S3_L004,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,SAMN41021637,Illumina Sequencing of Omni-C Libraries of HG0...,unaligned reads,paired,RANDOM,no shear
1,HG00126-1_FC2_S3_L004_R1_001.fastq.gz,fastq,HG00126-1_FC2_S3_L004,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,SAMN41021637,Illumina Sequencing of Omni-C Libraries of HG0...,unaligned reads,paired,RANDOM,no shear
2,HG00126-2_FC1_S4_L004_R1_001.fastq.gz,fastq,HG00126-2_FC1_S4_L004,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,SAMN41021637,Illumina Sequencing of Omni-C Libraries of HG0...,unaligned reads,paired,RANDOM,no shear
3,HG00126-2_FC2_S4_L004_R1_001.fastq.gz,fastq,HG00126-2_FC2_S4_L004,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,SAMN41021637,Illumina Sequencing of Omni-C Libraries of HG0...,unaligned reads,paired,RANDOM,no shear
4,HG00128-1_FC1_S9_L004_R1_001.fastq.gz,fastq,HG00128-1_FC1_S9_L004,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,SAMN41021641,Illumina Sequencing of Omni-C Libraries of HG0...,unaligned reads,paired,RANDOM,no shear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,HG01975-2_S4_L002_R1_001.fastq.gz,fastq,HG01975-2_S4_L002,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,SAMN33621947,Illumina Sequencing of Omni-C Libraries of HG0...,unaligned reads,paired,RANDOM,no shear
168,HG02841-1_S5_L001_R1_001.fastq.gz,fastq,HG02841-1_S5_L001,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,SAMN37797106,Illumina Sequencing of Omni-C Libraries of HG0...,unaligned reads,paired,RANDOM,no shear
169,HG02841-2_S5_L002_R1_001.fastq.gz,fastq,HG02841-2_S5_L002,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,SAMN37797106,Illumina Sequencing of Omni-C Libraries of HG0...,unaligned reads,paired,RANDOM,no shear
170,HG02984-1_S6_L001_R1_001.fastq.gz,fastq,HG02984-1_S6_L001,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,SAMN37797107,Illumina Sequencing of Omni-C Libraries of HG0...,unaligned reads,paired,RANDOM,no shear


In [77]:
hic_sra_merge_df.to_csv('sra_biosample_HIC_Y3_fillin_and_Y4_part1-redo.tsv',index=False, sep='\t')